In [1]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer

In [5]:
# Loading the data set
email_data = pd.read_csv("C:/Users/anjum/Documents/Exelir workshop/Dataset/sms_raw_NB.csv",encoding = "ISO-8859-1")

In [7]:
email_data

,type,text
0,ham,Hope you are having a good week. Just checking in
1,ham,K..give back my thanks.
2,ham,Am also doing in cbe only. But have to pay.
3,spam,"complimentary 4 STAR Ibiza Holiday or å£10,000..."
4,spam,okmail: Dear Dave this is your final notice to...
...,...,...
5554,ham,You are a great role model. You are giving so ...
5555,ham,"Awesome, I remember the last time we got someb..."
5556,spam,"If you don't, your prize will go to another cu..."
5557,spam,"SMS. ac JSco: Energy is high, but u may not kn..."


In [8]:
# cleaning data 
import re
stop_words = []

In [10]:
# Load the custom built Stopwords
with open("C:/Users/anjum/Documents/Exelir workshop/stopwords_en.txt","r") as sw:
    stop_words = sw.read()

In [11]:
stop_words = stop_words.split("\n")

In [12]:
def cleaning_text(i):
    i = re.sub("[^A-Za-z" "]+"," ",i).lower()
    i = re.sub("[0-9" "]+"," ",i)
    w = []
    for word in i.split(" "):
        if len(word)>3:
            w.append(word)
    return (" ".join(w))

In [13]:
# testing above function with sample text => removes punctuations, numbers
cleaning_text("Hope you are having a good week. Just checking in")
cleaning_text("hope i can understand your feelings 123121. 123 hi how .. are you?")
cleaning_text("Hi how are you, I am good")

'good'

In [14]:
email_data.text = email_data.text.apply(cleaning_text)


In [15]:
# removing empty rows
email_data = email_data.loc[email_data.text != " ",:]

In [16]:
# splitting data into train and test data sets 
from sklearn.model_selection import train_test_split

email_train, email_test = train_test_split(email_data, test_size = 0.2)

In [17]:
def split_into_words(i):
    return [word for word in i.split(" ")]

In [18]:
emails_bow = CountVectorizer(analyzer = split_into_words).fit(email_data.text)


In [19]:
all_emails_matrix = emails_bow.transform(email_data.text)


In [20]:
# For training messages
train_emails_matrix = emails_bow.transform(email_train.text)


In [21]:
# For testing messages
test_emails_matrix = emails_bow.transform(email_test.text)



In [22]:
# Learning Term weighting and normalizing on entire emails
tfidf_transformer = TfidfTransformer().fit(all_emails_matrix)



In [23]:
# Preparing TFIDF for train emails
train_tfidf = tfidf_transformer.transform(train_emails_matrix)
train_tfidf.shape # (row, column)



(4447, 6661)

In [24]:
# Preparing TFIDF for test emails
test_tfidf = tfidf_transformer.transform(test_emails_matrix)
test_tfidf.shape #  (row, column)

(1112, 6661)

In [25]:
from sklearn.naive_bayes import MultinomialNB as MB

In [26]:
# Multinomial Naive Bayes
classifier_mb = MB()
classifier_mb.fit(train_tfidf, email_train.type)



MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [27]:
# Evaluation on Test Data
test_pred_m = classifier_mb.predict(test_tfidf)
accuracy_test_m = np.mean(test_pred_m == email_test.type)
accuracy_test_m



0.9595323741007195

In [28]:
from sklearn.metrics import accuracy_score
accuracy_score(test_pred_m, email_test.type) 

pd.crosstab(test_pred_m, email_test.type)



type,ham,spam
row_0,,
ham,961,44
spam,1,106


In [29]:
# Training Data accuracy
train_pred_m = classifier_mb.predict(train_tfidf)
accuracy_train_m = np.mean(train_pred_m == email_train.type)
accuracy_train_m


0.9698673262873847